In [1]:
!pip install -q playwright nest_asyncio
!playwright install chromium
import nest_asyncio
import asyncio
import pandas as pd
from playwright.async_api import async_playwright
import re
nest_asyncio.apply()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 35.9 MB/s eta 0:00:00
172.5 MiB [] 0% 0.0s172.5 MiB [] 0% 352.0s172.5 MiB [] 0% 352.5s172.5 MiB [] 0% 225.3s172.5 MiB [] 0% 205.7s172.5 MiB [] 0% 158.7s172.5 MiB [] 0% 125.7s172.5 MiB [] 0% 128.5s172.5 MiB [] 0% 99.2s172.5 MiB [] 0% 78.3s172.5 MiB [] 0% 76.1s172.5 MiB [] 0% 59.6s172.5 MiB [] 0% 47.8s172.5 MiB [] 0% 45.2s172.5 MiB [] 1% 35.7s172.5 MiB [] 1% 29.5s172.5 MiB [] 1% 26.2s172.5 MiB [] 2% 20.6s172.5 MiB [] 2% 17.0s172.5 MiB [] 3% 14.8s172.5 MiB [] 4% 11.9s172.5 MiB [] 4% 10.0s172.5 MiB [] 5% 9.5s172.5 MiB [] 6% 8.6s172.5 MiB [] 7% 7.4s172.5 MiB [] 7% 7.3s172.5 MiB [] 8% 6.8s172.5 MiB [] 9% 6.2s172.5 MiB [] 9% 6.1s172.5 MiB [] 10% 5.7s172.5 MiB [] 11% 5.3s172.5 MiB [] 12% 5.3s172.5 MiB [] 12% 5.1s172.5 MiB [] 13% 4.7s172.5 MiB [] 14% 4.7s172.5 MiB [] 15% 4.5s172.5 MiB [] 16% 4.3s172.5 MiB [] 17% 4.1s172.5 MiB [] 18% 3.9s172.5 MiB [] 19% 3.8s172.5 MiB [] 20% 3.7s172.5 MiB [] 21% 3.6s172.5 MiB [] 21% 3.5s172.5 MiB [] 22% 3.

In [2]:
_BASE_URL_='https://www.zenga.hu'
def url_creator(place:str="pest",type:str="haz",price_min=40000000, price_max=100000000):

    return f"{_BASE_URL_}/{place}+elado+{type}+ar-{price_min}-{price_max}?page="

In [3]:
url_creator("biatorbagy")

'https://www.zenga.hu/biatorbagy+elado+haz+ar-40000000-100000000?page='

In [4]:
async def scrape_nof_links(start_url="https://www.zenga.hu/komarom-esztergom-megye+elado+haz?page=",page_num=1):
    '''
    Az megadott strat_url oldalán lévő linkek számát adja vissza
    :param start_url: az oldal url-je
    :param page_num: az oldal száma
    '''
    start_url = start_url+str(page_num)
    links = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(start_url)
        await page.wait_for_timeout(8000)

        # Scroll, hogy több kártya betöltődjön
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(6000)

        div_list = await page.query_selector_all("div[data-cy='search-page-total-count']") # linkek itt vannak a html-ben
        text=await div_list[0].text_content()
        print(f"Talált linkek száma: {text}")
        number_of_links = int(re.search(r'\d+', text.replace('\u00A0', '').replace(' ', '')).group())


        await browser.close()
        return number_of_links

In [5]:
number_of_links=await scrape_nof_links()
print(f"A keresésnek megfelelő linkek száma:{number_of_links}")

Talált linkek száma:  1 830 találat 
A keresésnek megfelelő linkek száma:1830


In [6]:
async def scrape_zenga_links(start_url="https://www.zenga.hu/komarom-esztergom-megye+elado+haz?page=",zenga_page=1)->list:
    '''
    A zenga.hu adott keresési oldaláról (oldalszám alapján) összegyűjti az ingatlanhirdetések
    kártyalinkjeit.

    Paraméterek
    ----------
        start_url : str, alapértelmezett: "https://www.zenga.hu/komarom-esztergom-megye+elado+haz?page="
            A listaoldal alapprefixe, amely "page=" végű query-vel zárul. A függvény a megadott
            oldalszámot fűzi a végére.
        zenga_page : int, alapértelmezett: 1
            A lekérdezni kívánt oldal száma.

    Visszatérés
    -----------
        list[str]
            A hirdetéskártyák abszolút URL-jeinek listája.
    '''

    start_url = start_url+str(zenga_page)
    links = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(start_url)
        await page.wait_for_timeout(8000)

        # Scroll, hogy több kártya betöltődjön
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(3000)

        a_links = await page.query_selector_all("a[data-cy='advert-card-link']") # linkek itt vannak a html-ben
        print(f"Talált linkek száma: {len(a_links)}")

        for a in a_links:
            href = await a.get_attribute("href")
            if href:
                links.append(_BASE_URL_ + "/" + href)

        await browser.close()
        return links

In [7]:
def get_all_url(place:str,prop_type="udulo-hetvegi-haz",l_price=40000000, h_price=100000000)->list:
    '''
    A megadott hely (place), ingatlantípus (prop_type) és ársáv (HUF) alapján összegyűjti a
    zenga.hu hirdetéslistáiról az összes hirdetés URL-jét. Először lekéri a találatok
    számát, majd az összes szükséges oldalon végiglépked és kimenti a linkeket.

    Paraméterek
    -----------
    place : str
        Település/megyenév vagy slug, amit a `url_creator` elfogad (pl. "komarom-esztergom-megye").
    prop_type : str, alapértelmezett: "haz"
        Ingatlantípus (pl. "haz", "lakas", "telek"). (Megj.: ütközik a beépített `type` névvel.)
    l_price : int, alapértelmezett: 40_000_000
        Alsó árhatár HUF-ban.
    h_price : int, alapértelmezett: 100_000_000
        Felső árhatár HUF-ban.

    Visszatérés
    -----------
    list[str]
        A talált hirdetések abszolút URL-jeinek listája.
    '''

    url=url_creator(place=place,type=prop_type,price_min=l_price,price_max=h_price)
    print(url)
    zenga_links=[]

    no_links = asyncio.run(scrape_nof_links(start_url=url,page_num=1))
    # Futtatás

    for i in range(-(-no_links // 20)):
        print(f"Page: {i+1}")
        zenga_page = asyncio.run( scrape_zenga_links(start_url=url,zenga_page=i+1))
        zenga_links.extend(zenga_page)
    # Kiírás
    for i, link in enumerate(zenga_links, 1):
        print(f"{i}. {link}")
    return zenga_links


In [8]:
zenga_links=get_all_url("biatorbagy","haz",40000000,100000000) # csak teszt

https://www.zenga.hu/biatorbagy+elado+haz+ar-40000000-100000000?page=
Talált linkek száma:  13 találat 
Page: 1
Talált linkek száma: 13
1. https://www.zenga.hu//ingatlan/elado-csaladi-haz-biatorbagy-torbagy/8661104?page=1&pos=1&cr=79
2. https://www.zenga.hu//ingatlan/elado-csaladi-haz-biatorbagy/8323033?page=1&pos=2&cr=20
3. https://www.zenga.hu//ingatlan/elado-csaladi-haz-biatorbagy/8595591?page=1&pos=3
4. https://www.zenga.hu//ingatlan/elado-csaladi-haz-biatorbagy/8656953?page=1&pos=4
5. https://www.zenga.hu//ingatlan/elado-csaladi-haz-biatorbagy/8655481?page=1&pos=5
6. https://www.zenga.hu//ingatlan/elado-hazresz-biatorbagy/8651573?page=1&pos=6
7. https://www.zenga.hu//ingatlan/elado-csaladi-haz-biatorbagy/8345664?page=1&pos=7
8. https://www.zenga.hu//ingatlan/elado-csaladi-haz-biatorbagy/8244737?page=1&pos=8
9. https://www.zenga.hu//ingatlan/elado-kuria-biatorbagy/7907575?page=1&pos=9
10. https://www.zenga.hu//ingatlan/elado-csaladi-haz-biatorbagy-bia/8059292?page=1&pos=10
11. http

In [57]:
city="balaton-deli-part+balaton-eszaki-part"
property="udulo-hetvegi-haz" # "udulo-hetvegi-haz", "haz"

In [58]:

zenga_links=get_all_url(city,property,10000000,40000000)
df=pd.DataFrame(zenga_links,columns=['link'])
csv_path='/content/drive/MyDrive/Zenga/'

https://www.zenga.hu/balaton-deli-part+balaton-eszaki-part+elado+udulo-hetvegi-haz+ar-10000000-40000000?page=
Talált linkek száma:  128 találat 
Page: 1
Talált linkek száma: 20
Page: 2
Talált linkek száma: 20
Page: 3
Talált linkek száma: 20
Page: 4
Talált linkek száma: 20
Page: 5
Talált linkek száma: 20
Page: 6
Talált linkek száma: 20
Page: 7
Talált linkek száma: 8
1. https://www.zenga.hu//ingatlan/elado-udulo-nyaralo-gyenesdias/8669458?page=1&pos=1&cr=95
2. https://www.zenga.hu//ingatlan/elado-kertes-haz-balatonkenese/8263900?page=1&pos=2&cr=55
3. https://www.zenga.hu//ingatlan/elado-udulo-nyaralo-balatonbereny/8596411?page=1&pos=3&cr=50
4. https://www.zenga.hu//ingatlan/elado-udulo-nyaralo-fonyod/8658241?page=1&pos=4&cr=42
5. https://www.zenga.hu//ingatlan/elado-udulo-nyaralo-balatonmariafurdo/8579431?page=1&pos=5&cr=21
6. https://www.zenga.hu//ingatlan/elado-udulo-nyaralo-balatonakarattya/8600951?page=1&pos=6&cr=15
7. https://www.zenga.hu//ingatlan/elado-udulo-nyaralo-siofok-balaton

In [59]:
df.to_csv(csv_path+f'links_{city}_{property}.csv',index=False)
# kimentettük a listát

In [12]:
raise SystemExit("🛑 Kód megszakítva ezen a ponton.") # A következő rész a lista feldolgozása

SystemExit: 🛑 Kód megszakítva ezen a ponton.

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [60]:
# visszaolvassuk a listát, mert ekkora adathalmazt nem feltétlen tudunk egy etapban feldolgozni
city="balaton-deli-part+balaton-eszaki-part"
property="udulo-hetvegi-haz" # "udulo-hetvegi-haz", "haz"
#^^ itt lehet megadni mi legyen a további feldolgozás alapja

csv_path='/content/drive/MyDrive/Zenga/'
df=pd.read_csv(csv_path+f'links_{city}_{property}.csv')
zenga_links=df.link.to_list()

In [61]:
_DEBUG_=True

In [62]:
async def scrape_ad_text(url):
    '''
    Egy zenga.hu hirdetés részletező oldaláról leolvassa a leírást és az árat és visszaadja egy listában.

    Paraméterek
    -----------
    url : str
        A hirdetés részletező oldalának abszolút URL-je.

    Visszatérés
    -----------
    list[dict[str, str]]
        Alapesetben egy elemű lista, eleme: {"url": <url>, "desc": <leírás>, "price": <ár>}.
        Hiba esetén a lista egy hibaüzenetet tartalmazó rekordot ad vissza
        (ld. Megjegyzések a kulcsnév-konzisztenciáról).
    '''

    results = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()


        try:
            if _DEBUG_:
                print(f"🔗 {url}")
            await page.goto(url, timeout=5000)  #20000 lehet csökkenteni?
            try:
                await page.get_by_text("Összes süti engedélyezése").click()
                if _DEBUG_:
                    print("Süti elfogadva")
            except:
                print("Süti gomb nem jelent meg")


            await page.wait_for_timeout(7000)
            #await page.screenshot(path="screenshot.png")  # DEBUG
            desc_element = await page.query_selector("span[data-cy='advert-details-description']")
            desc_text = await desc_element.inner_text() if desc_element else "Nincs szöveg"
            if _DEBUG_:
                print(f"Szöveg: {desc_text} ")

            price_element = await page.query_selector("div[data-cy='advert-details-price']")
            price_text = await price_element.inner_text() if price_element else "Nincs ár"
            if _DEBUG_:
                print(f"Ár: {price_text} ")

            #méret

            size_element = await page.query_selector("div[data-cy='advert-details-first-param']")
            size_text = await size_element.inner_text() if size_element else "Nincs méret"
            if _DEBUG_:
                print(f"Méret: {size_text} ")

            # telek mérete lot_size advert-details-second-param
            lot_size_element = await page.query_selector("div[data-cy='advert-details-second-param']")
            lot_size_text = await lot_size_element.inner_text() if lot_size_element else "Nincs telek méret"
            if _DEBUG_:
                print(f"Telek méret: {lot_size_text} ")

            # rooms szoák száma advert-details-third-param
            rooms_element = await page.query_selector("div[data-cy='advert-details-third-param']")
            rooms_text = await rooms_element.inner_text() if rooms_element else "Nincs szoba"
            if _DEBUG_:
                print(f"Szobák száma: {rooms_text} ")


            results.append({"url": url, "desc": desc_text, "price":price_text, "size":size_text, "lot_size":lot_size_text,"rooms":rooms_text})
        except Exception as e:
            if _DEBUG_:
                print(f"Hiba: {str(e)}")
            results.append({"url": url, "desc": f"Hiba: {str(e)}","price":f"Hiba: {str(e)}"})

        await browser.close()

    return results  #return a list


In [63]:
await scrape_ad_text("https://www.zenga.hu//ingatlan/elado-csaladi-haz-biatorbagy-bia/8059292?page=1&pos=8")

🔗 https://www.zenga.hu//ingatlan/elado-csaladi-haz-biatorbagy-bia/8059292?page=1&pos=8
Süti elfogadva
Szöveg: Nincs szöveg 
Ár: 95 millió Ft 
Méret: 98 m² 
Telek méret: 900 m² 
Szobák száma: 3 szoba 


[{'url': 'https://www.zenga.hu//ingatlan/elado-csaladi-haz-biatorbagy-bia/8059292?page=1&pos=8',
  'desc': 'Nincs szöveg',
  'price': '95\xa0millió Ft',
  'size': '98 m²',
  'lot_size': '900 m²',
  'rooms': '3 szoba'}]

In [64]:
city

'balaton-deli-part+balaton-eszaki-part'

In [65]:
def scrape_zenga_list(z_list):
    """
    A megadott zenga.hu hirdetés-URL-ek listáját sorban feldolgozza: mindegyikre lefuttatja a
    `scrape_ad_text` aszinkron függvényt.

    Paraméterek
    -----------
    z_list : list[str]
        Hirdetés-részletező oldalak abszolút URL-jeinek listája.

    Visszatérés
    -----------
    list[dict[str, str]]
        A hirdetésekből kinyert rekordok listája. Az egyes elemek a `scrape_ad_text` által
        visszaadott dict-ek (pl. {"url", "desc", "price"}.

    """
    i=0
    out_list=[]
    for url in z_list:
        url_data = asyncio.run(scrape_ad_text(url))
        if _DEBUG_:
            print(url_data)
        print(f"{i+1}/{len(z_list)}",end="_")
        i+=1
        if i%10==0:
            print()
        out_list.extend(url_data)
    return out_list

In [66]:
_DEBUG_=False

In [67]:
nl=1000
for i in range(0000, len(zenga_links), nl):
    zenga_data=scrape_zenga_list(zenga_links[i:i+nl])
    df=pd.DataFrame(zenga_data)
    csv_path='/content/drive/MyDrive/Zenga/'
    df.to_csv(csv_path+f'zenga_data_{city}_{property}_{str(i)}_{str(i+nl-1)}.csv',index=False)

1/128_2/128_3/128_4/128_5/128_6/128_7/128_8/128_9/128_10/128_
11/128_12/128_13/128_14/128_15/128_16/128_17/128_18/128_19/128_20/128_
21/128_22/128_23/128_24/128_25/128_26/128_27/128_28/128_29/128_30/128_
31/128_32/128_33/128_34/128_35/128_36/128_37/128_38/128_39/128_40/128_
41/128_42/128_43/128_44/128_45/128_46/128_47/128_48/128_49/128_50/128_
51/128_52/128_53/128_54/128_55/128_56/128_57/128_58/128_59/128_60/128_
61/128_62/128_63/128_64/128_65/128_66/128_67/128_68/128_69/128_70/128_
71/128_72/128_73/128_74/128_75/128_76/128_77/128_78/128_79/128_80/128_
81/128_82/128_83/128_84/128_85/128_86/128_87/128_88/128_89/128_90/128_
91/128_92/128_93/128_94/128_95/128_96/128_97/128_98/128_99/128_100/128_
101/128_102/128_103/128_104/128_105/128_106/128_107/128_108/128_109/128_110/128_
111/128_112/128_113/128_114/128_115/128_116/128_117/128_118/128_119/128_120/128_
121/128_122/128_123/128_124/128_125/128_126/128_127/128_128/128_

In [68]:
#szoveg=df.desc[0]

In [69]:
raise SystemExit("🛑 Kód megszakítva ezen a ponton.") # A következő rész a AI prompt beállításait tartalmazza

SystemExit: 🛑 Kód megszakítva ezen a ponton.

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#ujraépítjuk az adatokat, innen bármikor el kell indulni
city="komarom_esztergom_megye"
csv_path='/content/drive/MyDrive/Zenga/'
df_city=pd.read_csv(csv_path+f'zenga_data_{city}.csv')

In [ ]:
from openai import OpenAI
import json

In [ ]:
from google.colab import userdata
key=userdata.get('OpenAi_API')

In [ ]:
#Demo szöveg
szoveg='''
Budafok szép panorámás részén a Rózsavölgy feletti dombon, csendes mellékutcában eladó egy bájos kis ház 520 négyzetméteres telken.

A kis ház 35 négyzetméteres, egy légtérben van kialakítva benne a nappali, melyet részben nyitott válaszfallal szeparáltak el a konyha-étkező résztől. A zuhanyzós fürdő és a WC egy helyiségben található.

A víz, az áram és az internet be van kötve a házba. Gáz és csatorna az utcában. A csatorna bevezetését már kezdeményezte a szomszédos telek tulajdonosa.

A telken áll egy másik kis házikó is, ami jelenleg egy 9 négyzetméteres szobaként funkcionál.

A kert szép és gondozott, sok gyümölcsfával. Nagyobb mérete miatt sok lehetőséget rejt. Kialakítható rajta több nagy magaságyás, építhető kerti medence vagy finn szauna is.

További információkért és időpont egyeztetésért kérem hívjon telefononn.
Hitellel vásárlás esetén bankfüggetlen és DÍJMENTES hiteltanácsadó közreműködését biztosítjuk, aki a teljes folyamaton végigkíséri Önt.
Az adásvétel biztonságos és gördülékeny bonyolítása érdekében tapasztalt ingatlan szakjogászaink szolgáltatását is ajánlom.
'''

In [ ]:
szoveg=df_city.desc[0]


print(szoveg)
print(df_city.url[0])

In [ ]:
def create_prompt(szoveg):
    prompt_message=[
            {"role": "system",
             "content": f'''
                    Te egy ingatlanhirdetés-elemző nyelvész vagy, komoly ingatlanhirdetési tapasztalattal.
                    Célod, hogy az ingatlan hirdetések tökéletesek legyenek ezért kidolgoztál egy hirdetés értékelési szempontrendszert.
                    Ezek a szempontok alapján tökéletes értékelést tudsz adni az adott hirdetés szövege alapján.
                    A szempontok:
                        1. Érthetőség
                        2. Részletesség / információtartalom
                        3. Szerkezet, logikai felépítés
                        4. Célcsoport megszólítása
                        5. Stílus és nyelvhelyesség
                        6. Előnyök kiemelése
                        7. Negatívumok őszinte kezelése
                        8. Eladásra ösztönzés'''},
            {"role": "user",
             "content": """
                            Értékeld az alábbi ingatlanhirdetés szöveget és az értékelést a scoring mezőbe helyezzed el.
                            Az alábbi szempontok alapján 1–5-ig pontozzad, és hozz létre egy összesített eredményt  1-8 közötti szempontok értékelésének átlagaként ez legyen a 9. Összesítés :
                            Majd a tudásod alapján adj egy alternatív javaslatot az eredeti szöveg javítására, hogy a lehető legjobban megfeleljen a szempontoknak.
                            Az alternatív javaslatot strukturáld, és tördeld a jobb érthetőség érdekében, de markdown formátumot ne használj!
                            Fontos, hogy a saját szempontrendszered szerint az alternatív javaslatod értékelése jobb legyen az eredeti értékelésnél!
                            A válaszod csak érvényes JSON formátumban legyen, pontosan az alábbi struktúrában:

                            {"scoring":
                                {

                                    "Érthetőség": <szám>,
                                    "Részletesség": <szám>,
                                    "Szerkezet": <szám>,
                                    "Célcsoport": <szám>,
                                    "Stílus": <szám>,
                                    "Előnyök": <szám>,
                                    "Negatívumok": <szám>,
                                    "Ösztönzés": <szám>,
                                    "Összesítés": <szám.tizedes>"
                                },
                            "proposal":"Ide kerüljön a javított szöveg javaslatod"
                            }


                    A HIRDETÉS SZÖVEGE:
                        """+f" {szoveg}"}
        ]
    return prompt_message


In [ ]:
prompt_message=create_prompt(szoveg)


In [ ]:
def get_response(szoveg:str):
    '''
    A megadott szöveghez promptot készít (`create_prompt`), elküldi az OpenAI Chat
    Completions API-nak (gpt-4o), majd a választ JSON-ként beolvassa és dict-ként visszaadja.

    Paraméterek
    -----------
    szoveg : str
        A bemeneti szöveg, amelyből a `create_prompt` összeállítja a `messages` listát.

    Visszatérés
    -----------
    dict
        A modell által visszaadott JSON objektum, Python dict-be parse-olva.
    '''
    client = OpenAI(api_key=key)
    prompt_message=create_prompt(szoveg)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=prompt_message,
        temperature=0.5,
        max_tokens=7000
    )

    content = response.choices[0].message.content

    # a gpt-4 így adja vissza a json választ, kiszedjük a jsont a felesleges keretből
    if content.startswith("```json"):
        content = content.strip("`").lstrip("json").strip()

    #
    out_dict = json.loads(content)

    return(out_dict)

In [ ]:
resp_data=get_response(prompt_message)

In [ ]:
print(resp_data)

In [ ]:
resp_data["scoring"]

In [ ]:
resp_data["proposal"]